<a href="https://colab.research.google.com/github/sankalpachowdhury/Candidate_Skill_Assessment_using_Graph_Neural_Network/blob/main/GIKT_Tf_version_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Framework

In [ ]:
# Run only if needed
!pip install tensorflow==2 #1.13.0rc1
import tensorflow as tf
print(tf.__version__)

2.0.0


In [ ]:
# run to check version
import tensorflow as tf
print(tf.__version__)

2.5.0


In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
# gikt tf v2 new commit
#https://github.com/ajenningsfrankston/graph_kt.git
!git clone "https://github.com/ajenningsfrankston/graph_kt"

Cloning into 'graph_kt'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 28 (delta 7), reused 27 (delta 6), pack-reused 0
Unpacking objects: 100% (28/28), done.


In [ ]:
# import all codes
!git clone "https://github.com/Rimoku/GIKT.git"


Cloning into 'GIKT'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 19 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.


In [ ]:

# import codes from own github 

#DATASET

In [ ]:
#get the data from the gdrive

# 1 mount gdrive
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
# 2 copy the folders to local
!cp -r /content/gdrive/MyDrive/FinalYear_Project_Dataset/data /content/data/

In [ ]:
# create logs and checkpoint folders
!mkdir logs
!mkdir checkpoint

#Convert to Tf v2

In [ ]:
!tf_upgrade_v2 -h

usage: tf_upgrade_v2 [-h] [--infile INPUT_FILE] [--outfile OUTPUT_FILE]
                     [--intree INPUT_TREE] [--outtree OUTPUT_TREE]
                     [--copyotherfiles COPY_OTHER_FILES] [--inplace]
                     [--import_rename] [--reportfile REPORT_FILENAME]
                     [--mode {DEFAULT,SAFETY}] [--print_all]

Convert a TensorFlow Python file from 1.x to 2.0

Simple usage:
  tf_upgrade_v2.py --infile foo.py --outfile bar.py
  tf_upgrade_v2.py --infile foo.ipynb --outfile bar.ipynb
  tf_upgrade_v2.py --intree ~/code/old --outtree ~/code/new

optional arguments:
  -h, --help            show this help message and exit
  --infile INPUT_FILE   If converting a single file, the name of the file to
                        convert
  --outfile OUTPUT_FILE
                        If converting a single file, the output filename.
  --intree INPUT_TREE   If converting a whole tree of files, the directory to
                        read from (relative or absolute).
  --ou

In [ ]:
!tf_upgrade_v2 \
    --intree GIKT/ \
    --outtree GIKT_v2/ \
    --reportfile tree_report.txt

INFO line 10:17: Renamed 'tf.ConfigProto' to 'tf.compat.v1.ConfigProto'
INFO line 13:9: Renamed 'tf.Session' to 'tf.compat.v1.Session'
INFO line 19:16: Renamed 'tf.train.Saver' to 'tf.compat.v1.train.Saver'
INFO line 25:21: Renamed 'tf.global_variables_initializer' to 'tf.compat.v1.global_variables_initializer'
WARNING line 158:4: *.save requires manual check. (This warning is only applicable if the code saves a tf.Keras model) Keras model.save now saves to the Tensorflow SavedModel format by default, instead of HDF5. To continue saving to HDF5, add the argument save_format='h5' to the save() function.
INFO line 48:13: Renamed 'tf.variable_scope' to 'tf.compat.v1.variable_scope'
WARNING line 49:27: tf.get_variable requires manual check. tf.get_variable returns ResourceVariables by default in 2.0, which have well-defined semantics and are stricter about shapes. You can disable this behavior by passing use_resource=False, or by calling tf.compat.v1.disable_resource_variables().
INFO line

#*Test* Drive

In [ ]:
#/content/GIKT_v2/main.py \
!python /content/graph_kt/main.py --dataset /content/data/assist09_3/assist09_3 \
--n_hop 3 \
--log_dir /content/logs \
--checkpoint_dir /content/checkpoint \
--skill_neighbor_num 4 \
--question_neighbor_num 4 \
--hist_neighbor_num 3 \
--next_neighbor_num 4 \
--model hsei \
--lr 0.001 \
--att_bound 0.7 \
--sim_emb question_emb \
--dropout_keep_probs [0.8,0.8,1]


2021-07-17 18:06:30.791642: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
hsei
{'data_dir': 'data', 'log_dir': '/content/logs', 'train': 1, 'hidden_neurons': [200, 100], 'lr': 0.001, 'lr_decay': 0.92, 'checkpoint_dir': '/content/checkpoint', 'dropout_keep_probs': '[0.8,0.8,1]', 'aggregator': 'sum', 'model': 'hsei', 'l2_weight': 1e-08, 'limit_max_len': 200, 'limit_min_len': 3, 'dataset': '/content/data/assist09_3/assist09_3', 'field_size': 3, 'embedding_size': 100, 'max_step': 200, 'input_trans_size': 100, 'batch_size': 32, 'select_index': [0, 1, 2], 'num_epochs': 150, 'n_hop': 3, 'skill_neighbor_num': 4, 'question_neighbor_num': 4, 'hist_neighbor_num': 3, 'next_neighbor_num': 4, 'att_bound': 0.7, 'sim_emb': 'question_emb', 'tag': 1626545192.6224916}
original test seqs num:893
167
17737
2021-07-17 18:06:40.247804: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI De

#Modules unit testing

In [ ]:
#aggregators.py
import tensorflow as tf
from abc import abstractmethod

LAYER_IDS = {}


def get_layer_id(layer_name=''):
    if layer_name not in LAYER_IDS:
        LAYER_IDS[layer_name] = 0
        return 0
    else:
        LAYER_IDS[layer_name] += 1
        return LAYER_IDS[layer_name]


class Aggregator(object):
    def __init__(self, batch_size, seq_len,dim, dropout, act, name):
        if not name:
            layer = self.__class__.__name__.lower()
            name = layer + '_' + str(get_layer_id(layer))
        self.name = name
        self.dropout = dropout
        self.act = act
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.dim = dim

    def __call__(self, self_vectors, neighbor_vectors, question_embeddings):
        outputs = self._call(self_vectors, neighbor_vectors, question_embeddings)
        return outputs

    @abstractmethod
    def _call(self, self_vectors, neighbor_vectors, question_embeddings):
        # dimension:
        # self_vectors: [batch_size, -1, dim]
        # neighbor_vectors: [batch_size, -1, n_neighbor, dim]
        # neighbor_relations: [batch_size, -1, n_neighbor, dim]
        # user_embeddings: [batch_size, dim]
        pass




class SumAggregator(Aggregator):
    def __init__(self, batch_size, seq_len,dim, dropout=0., act=tf.nn.relu, name=None):
        super(SumAggregator, self).__init__(batch_size,seq_len,dim, dropout, act, name)

        with tf.variable_scope(self.name):
            self.weights = tf.get_variable(
                shape=[self.dim, self.dim], initializer=tf.contrib.layers.xavier_initializer(), name='weights')
            self.bias = tf.get_variable(shape=[self.dim], initializer=tf.zeros_initializer(), name='bias')

    def _call(self,self_vectors, neighbor_vectors, question_embeddings):
        # [batch_size,seq_len, -1, dim]
        neighbors_agg = tf.reduce_mean(neighbor_vectors, axis=-2)
        output = tf.reshape(self_vectors + neighbors_agg, [-1, self.dim])
        #neighbors_agg = tf.concat([tf.reshape(self_vectors,[self.batch_size,self.seq_len,-1,1,self.dim]),neighbor_vectors],-2)

        # [-1, dim]
        #output = tf.reshape(tf.reduce_mean(neighbors_agg,-2), [-1, self.dim])
        #output = tf.nn.dropout(output, keep_prob=self.dropout)
        output = tf.nn.dropout(output, rate=1 - keep_prob)
        output = tf.matmul(output, self.weights) + self.bias

        # [batch_size,seq_len, -1, dim]
        output = tf.reshape(output, [self.batch_size,self.seq_len, -1, self.dim])

        return self.act(output)



class ConcatAggregator(Aggregator):
    def __init__(self, batch_size,seq_len, dim, dropout=0., act=tf.nn.relu, name=None):
        super(ConcatAggregator, self).__init__(batch_size, seq_len,dim, dropout, act, name)

        with tf.variable_scope(self.name):
            self.weights = tf.get_variable(
                shape=[self.dim * 2, self.dim], initializer=tf.contrib.layers.xavier_initializer(), name='weights')
            self.bias = tf.get_variable(shape=[self.dim], initializer=tf.zeros_initializer(), name='bias')

    def _call(self, self_vectors, neighbor_vectors, question_embeddings):
        # [batch_size,seq_len, -1, dim]
        neighbors_agg = tf.reduce_mean(neighbor_vectors, axis=-2)

        # [batch_size,seq_len, -1, dim * 2]
        output = tf.concat([self_vectors, neighbors_agg], axis=-1)

        # [-1, dim * 2]
        output = tf.reshape(output, [-1, self.dim * 2])
        #output = tf.nn.dropout(output, keep_prob=1-self.dropout)
        output = tf.nn.dropout(output, rate=keep_prob)

        # [-1, dim]
        output = tf.matmul(output, self.weights) + self.bias

        # [batch_size, -1, dim]
        output = tf.reshape(output, [self.batch_size,self.seq_len, -1, self.dim])

        return self.act(output)




In [ ]:
# model.py

# encoding:utf-8
import tensorflow as tf
#from aggregators import SumAggregator, ConcatAggregator


class GIKT(object):
    def __init__(self, args):

        self.args = args
        self.hidden_neurons = args.hidden_neurons
        self.max_step = args.max_step - 1
        self.feature_answer_size = args.feature_answer_size
        self.field_size = args.field_size
        self.embedding_size = args.embedding_size

        self.dropout_keep_probs = eval(args.dropout_keep_probs)
        self.select_index = args.select_index
        self.hist_neighbor_num = args.hist_neighbor_num  # M
        self.next_neighbor_num = args.next_neighbor_num  # N
        self.lr = args.lr
        self.n_hop = args.n_hop

        self.question_neighbor_num = args.question_neighbor_num
        self.skill_neighbor_num = args.skill_neighbor_num

        self.question_neighbors = args.question_neighbors
        self.skill_neighbors = args.skill_neighbors

        self.keep_prob = tf.compat.v1.placeholder(tf.float32)  # dropout keep prob
        self.keep_prob_gnn = tf.compat.v1.placeholder(tf.float32)  # dropout keep prob
        self.is_training = tf.compat.v1.placeholder(tf.bool)
        self.features_answer_index = tf.compat.v1.placeholder(tf.int32, [None, self.max_step + 1, self.field_size])
        self.target_answers = tf.compat.v1.placeholder(tf.float32, [None, self.max_step])
        self.sequence_lens = tf.compat.v1.placeholder(tf.int32, [None])
        self.hist_neighbor_index = tf.compat.v1.placeholder(tf.int32, [None, self.max_step, self.hist_neighbor_num])
        self.batch_size = tf.shape(self.features_answer_index)[0]
        self.feature_embedding = tf.compat.v1.get_variable("feature_embedding", [self.feature_answer_size, self.embedding_size],initializer=tf.keras.initializers.GlorotUniform())

        if args.aggregator == 'sum':
            self.aggregator_class = SumAggregator
        elif args.aggregator == 'concat':
            self.aggregator_class = ConcatAggregator
        else:
            raise Exception("Unknown aggregator: " + args.aggregator)
        self.build_model()

    def build_model(self):
        hidden_size = self.hidden_neurons[-1]
        select_feature_index = tf.gather(self.features_answer_index, self.select_index, axis=-1)
        select_size = len(self.select_index)
        questions_index = select_feature_index[:, :-1, 1]
        next_questions_index =select_feature_index[:,1:,1]
        skill_index = select_feature_index[:,:-1,0]
        next_skill_index =select_feature_index[:,1:,0]

        self.input_questions_embedding = tf.nn.embedding_lookup(self.feature_embedding, questions_index) #[batch_size,seq_len,d]
        self.next_questions_embedding = tf.nn.embedding_lookup(self.feature_embedding,next_questions_index) #[batch_size,seq_len,select_size-1,d]

        self.input_skills_embedding = tf.nn.embedding_lookup(self.feature_embedding, skill_index) #[batch_size,seq_len,d]
        self.next_skills_embedding = tf.nn.embedding_lookup(self.feature_embedding,next_skill_index) #[batch_size,seq_len,select_size-1,d]


        input_answers_embedding = tf.nn.embedding_lookup(self.feature_embedding,select_feature_index[:,:-1,-1]) #[batch_size,seq_len,1,d]
        input_answers_index = select_feature_index[:,:-1,-1]

        if self.n_hop>0:
            #gnn
            input_neighbors = self.get_neighbors(self.n_hop,questions_index)##[[batch_size,seq_len],[batch_size,seq_len,q_neighbor_num],[batch_size,seq_len,q_neighbor_num*s_neighbor_num]
            aggregate_embedding,self.aggregators = self.aggregate(input_neighbors, self.input_questions_embedding)

            next_input_neighbors = self.get_neighbors(self.n_hop,next_questions_index)##[[batch_size,seq_len],[batch_size,seq_len,q_neighbor_num],[batch_size,seq_len,q_neighbor_num*s_neighbor_num]
            next_aggregate_embedding,self.aggregators = self.aggregate(next_input_neighbors, self.next_questions_embedding)

            feature_emb_size =  self.embedding_size
            feature_trans_embedding  = tf.reshape(tf.keras.layers.Dense(tf.reshape(aggregate_embedding[0],[-1,feature_emb_size]),hidden_size, activation = None, name = 'feature_layer', reuse = False), [-1,self.max_step, hidden_size]) 
            #feature_trans_embedding  = tf.reshape(tf.keras.layers.Dense(tf.reshape(aggregate_embedding[0],[-1,feature_emb_size]),hidden_size, activation = tf.nn.relu, name = 'feature_layer', reuse = False), [-1,self.max_step, hidden_size]) #[batch_size,max_step,hidden_size]
            next_trans_embedding  = tf.reshape(tf.keras.layers.Dense(tf.reshape(next_aggregate_embedding[0],[-1,feature_emb_size]),hidden_size, activation = 'relu', name = 'feature_layer', reuse = True), [-1,self.max_step, hidden_size]) #[batch_size,max_step,hidden_size]

        else:
            feature_emb_size =  self.embedding_size
            feature_trans_embedding  = tf.reshape(tf.compat.v1.layers.dense(tf.reshape(self.input_questions_embedding,[-1,feature_emb_size]),hidden_size, activation = tf.nn.relu(), name = 'feature_layer', reuse = False), [-1,self.max_step, hidden_size]) #[batch_size,max_step,hidden_size]
            next_trans_embedding  = tf.reshape(tf.layers.dense(tf.reshape(self.next_questions_embedding,[-1,feature_emb_size]),hidden_size, activation = tf.nn.relu(), name = 'feature_layer', reuse = True), [-1,self.max_step, hidden_size]) #[batch_size,max_step,hidden_size]

            # # gnn
            input_neighbors = self.get_neighbors(1,
                                                 questions_index)  ##[[batch_size,seq_len],[batch_size,seq_len,q_neighbor_num],[batch_size,seq_len,q_neighbor_num*s_neighbor_num]

            next_input_neighbors = self.get_neighbors(1,
                                                      next_questions_index)  ##[[batch_size,seq_len],[batch_size,seq_len,q_neighbor_num],[batch_size,seq_len,q_neighbor_num*s_neighbor_num]

            next_aggregate_embedding = [next_trans_embedding,tf.reshape(tf.gather(self.feature_embedding, tf.reshape(next_input_neighbors[-1], [-1])),
                                            [self.batch_size, self.max_step, -1, self.embedding_size])]
            aggregate_embedding = [feature_trans_embedding,tf.reshape(tf.gather(self.feature_embedding, tf.reshape(input_neighbors[-1], [-1])),
                                            [self.batch_size, self.max_step, -1, self.embedding_size])]





        input_fa_embedding = tf.reshape(tf.concat([feature_trans_embedding,input_answers_embedding],-1),[-1,hidden_size+self.embedding_size]) #embedding_size*2
        input_trans_embedding = tf.reshape(tf.layers.dense(input_fa_embedding, hidden_size),
                                            [-1, self.max_step, hidden_size])


        #create rnn cell
        hidden_layers = []
        for idx, hidden_size in enumerate(self.hidden_neurons):
            lstm_layer = tf.compat.v1.nn.rnn_cell.BasicLSTMCell(num_units=hidden_size,name='input_rnn%d'%idx)
            hidden_layer = tf.compat.v1.nn.rnn_cell.DropoutWrapper(cell=lstm_layer,
                                                         output_keep_prob=self.keep_prob)
            hidden_layers.append(hidden_layer)
        self.hidden_cell = tf.compat.v1.nn.rnn_cell.MultiRNNCell(cells=hidden_layers, state_is_tuple=True)  # RNN

        output_series = []
        self.state = self.hidden_cell.zero_state(self.batch_size, tf.float32)

        for i in range(self.max_step):
            current_output, self.state = self.hidden_cell(input_trans_embedding[:, i, :], self.state)
            output_series.append(current_output)

        output_series = tf.reshape(tf.concat(output_series, 1), [-1, self.max_step, hidden_size])


        if self.args.model == "hssi":
            self.hist_neighbors_features = self.hist_neighbor_sampler(
                output_series)  # [self.batch_size,max_step,self.hist_neighbor_num,hidden_size]
        elif self.args.model == "hsei":
            self.hist_neighbors_features = self.hist_neighbor_sampler(
                input_trans_embedding)  # [self.batch_size,max_step,self.hist_neighbor_num,hidden_size]
        elif self.args.model == "ssei":
            if self.args.sim_emb == "skill_emb":
                self.hist_neighbors_features = self.hist_neighbor_sampler1(self.input_skills_embedding,
                                                                           self.next_skills_embedding,
                                                                           input_trans_embedding)  # [self.batch_size,max_step,self.hist_neighbor_num,hidden_size]
            elif self.args.sim_emb == "question_emb":
                self.hist_neighbors_features = self.hist_neighbor_sampler1(self.input_questions_embedding,
                                                                           self.next_questions_embedding,
                                                                           input_trans_embedding)  # [self.batch_size,max_step,self.hist_neighbor_num,hidden_size]
            else:
                self.hist_neighbors_features = self.hist_neighbor_sampler1(feature_trans_embedding,
                                                                           next_trans_embedding,
                                                                           input_trans_embedding)  # [self.batch_size,max_step,self.hist_neighbor_num,hidden_size]
        else:
            if self.args.sim_emb == "skill_emb":
                self.hist_neighbors_features = self.hist_neighbor_sampler1(self.input_skills_embedding,
                                                                           self.next_skills_embedding,
                                                                           output_series)  # [self.batch_size,max_step,self.hist_neighbor_num,hidden_size]
            elif self.args.sim_emb == "question_emb":
                self.hist_neighbors_features = self.hist_neighbor_sampler1(self.input_questions_embedding,
                                                                           self.next_questions_embedding,
                                                                           output_series)  # [self.batch_size,max_step,self.hist_neighbor_num,hidden_size]
            else:
                self.hist_neighbors_features = self.hist_neighbor_sampler1(feature_trans_embedding,
                                                                           next_trans_embedding,
                                                                           output_series)  # [self.batch_size,max_step,self.hist_neighbor_num,hidden_size]

        if self.next_neighbor_num!=0:
            Nn = self.next_neighbor_sampler(next_aggregate_embedding)  # [batch_size,max_step,N+1,embedding_size]
            Nn = tf.concat([tf.expand_dims(next_trans_embedding,2),Nn],-2)
            next_neighbor_num = self.next_neighbor_num+1
        else:
            Nn = tf.expand_dims(next_trans_embedding, 2)
            next_neighbor_num = 1


        if self.hist_neighbor_num != 0:


            Nh = tf.concat([tf.expand_dims(output_series, 2), self.hist_neighbors_features],
                           2)  # [self.batch_size,max_step,M+1,feature_trans_size]]

            logits = tf.reduce_sum(tf.expand_dims(Nh, 3) * tf.expand_dims(Nn, 2),
                                   axis=4)  # [-1,max_step,Nh,1,emb_size]*[-1,max_step,1,Nn,emb_size]



            logits = tf.reshape(logits, [-1, self.max_step, (
                        self.hist_neighbor_num + 1) * next_neighbor_num])  # ====>[batch_size,max_step,Nu*Nv]


        else:

            Nh = tf.expand_dims(output_series, 2)  # [self.batch_size,max_step,1,feature_trans_size]


            logits = tf.reduce_sum(tf.expand_dims(Nh, 3) * tf.expand_dims(Nn, 2),
                                   axis=4)  # [-1,max_step,Nh,1,emb_size]*[-1,max_step,1,Nn,emb_size]

            logits = tf.reshape(logits,
                                [-1, self.max_step, 1 * next_neighbor_num])  # ====>[batch_size,max_step,Nu*Nv]






        with tf.variable_scope('ni'):
            w1 = tf.get_variable('atn_weights_1',[hidden_size, 1], initializer=tf.contrib.layers.xavier_initializer())
            w2 = tf.get_variable('atn_weights_2',[hidden_size, 1],initializer=tf.contrib.layers.xavier_initializer())
            b1 = tf.get_variable('atn_bias_1',[1],initializer=tf.zeros_initializer())
            b2 = tf.get_variable('atn_bias_2',[1],initializer=tf.zeros_initializer())
        if select_size > 3:

            f1 = tf.reshape(tf.matmul(tf.reshape(Nh, [-1, hidden_size]), w1) + b1,
                            [-1, self.max_step, self.hist_neighbor_num + 1, 1])
            f2 = tf.reshape(tf.matmul(tf.reshape(Nn, [-1, hidden_size]), w2) + b2,
                            [-1, self.max_step, 1, next_neighbor_num])
            coefs = tf.nn.softmax(tf.nn.tanh(
                tf.reshape(f1 + f2, [-1, self.max_step, (self.hist_neighbor_num + 1) * next_neighbor_num])))  # temp=10
        else:
            f1 = tf.reshape(tf.matmul(tf.reshape(Nh, [-1, hidden_size]), w1) + b1,
                            [-1, self.max_step, self.hist_neighbor_num + 1, 1])
            f2 = tf.reshape(tf.matmul(tf.reshape(Nn, [-1, hidden_size]), w2) + b2,
                            [-1, self.max_step, 1, next_neighbor_num])
            coefs = tf.nn.softmax(tf.nn.tanh(
                tf.reshape(f1 + f2, [-1, self.max_step, (self.hist_neighbor_num + 1) * next_neighbor_num])))  # temp=10

        #coefs = tf.nn.softmax(logits)
        self.logits = tf.reduce_sum(logits * coefs, axis=-1)

        self.flat_target_logits = flat_target_logits = tf.reshape(self.logits, [-1])
        self.flat_target_correctness = tf.reshape(self.target_answers, [-1])
        self.pred = tf.sigmoid(tf.reshape(flat_target_logits, [-1, self.max_step]))
        self.binary_pred = tf.cast(tf.greater_equal(self.pred, 0.5), tf.int32)

        self.filling_seqs = tf.cast(tf.sequence_mask(self.sequence_lens - 1, self.max_step),
                                    dtype=tf.float32)  # [batch_size,seq_len]
        index = tf.where(tf.not_equal(tf.reshape(self.filling_seqs, [-1]), tf.constant(0, dtype=tf.float32)))
        clear_flat_target_logits = tf.gather(self.flat_target_logits, index)
        clear_flat_target_correctness = tf.gather(self.flat_target_correctness, index)
        self.loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=clear_flat_target_correctness,
                                                                          logits=clear_flat_target_logits))



        self.global_step = tf.Variable(0, name="global_step", trainable=False)
        # self.lr = tf.Variable(0.0, trainable=False)

        trainable_vars = tf.trainable_variables()
        self.grads, _ = tf.clip_by_global_norm(tf.gradients(self.loss, trainable_vars), 50)
        # optimizer = tf.train.AdamOptimizer(learning_rate=self.lr)
        # optimizer = tf.train.GradientDescentOptimizer(self.lr)
        # optimizer = tf.train.MomentumOptimizer(learning_rate=self.lr,momentum=0.95)
        # self.train_op = optimizer.apply_gradients(zip(self.grads, trainable_vars))
        self.train_op = tf.train.AdamOptimizer(learning_rate=self.lr,
                                               beta1=0.9, beta2=0.999, epsilon=1e-8). \
            minimize(self.loss, global_step=self.global_step)

        print("initialize complete")



    def hist_neighbor_sampler(self,input_embedding):
        zero_embeddings = tf.expand_dims(tf.zeros([self.batch_size,self.hidden_neurons[-1]],dtype=tf.float32),1)#[batch_size,1,hidden_size]
        input_embedding = tf.concat([input_embedding,zero_embeddings],1)#[batch_size,max_step+1,hidden_size]
        #input_embedding:[batch_size,max_step,fa_trans_size]
        temp_hist_index = tf.reshape(self.hist_neighbor_index,[-1,self.max_step*self.hist_neighbor_num]) #[self.batch_size, max_step*M]
        hist_neighbors_features =tf.reshape(tf.batch_gather(input_embedding,temp_hist_index),[-1,self.max_step,self.hist_neighbor_num,input_embedding.shape[-1]])

        #select last neigbor_num questions

        return hist_neighbors_features

    def hist_neighbor_sampler1(self, input_q_emb, next_q_emb, qa_emb):#sample based on question similarity
        #next_q_emb:[batch_size,ms,emb_size]
        mold_nextq = tf.sqrt(tf.reduce_sum(next_q_emb*next_q_emb,-1))#[bs,ms]
        next_q_emb = tf.expand_dims(next_q_emb,2)
        mold_inputq = tf.sqrt(tf.reduce_sum(input_q_emb*input_q_emb,-1))#[bs,ms]
        input_q_emb = tf.expand_dims(input_q_emb,1)
        q_similarity = tf.reduce_sum(next_q_emb*input_q_emb,-1)#[batch_size,ms,ms]
        molds = tf.expand_dims(mold_nextq,2)*tf.expand_dims(mold_inputq,1)#[bs,ms,ms]
        q_similarity = q_similarity/molds


        zero_embeddings = tf.expand_dims(tf.zeros([self.batch_size,self.hidden_neurons[-1]],dtype=tf.float32),1)#[batch_size,1,hidden_size]
        qa_emb = tf.concat([qa_emb, zero_embeddings], 1)#[batch_size,max_step+1,hidden_size]
        paddings = tf.fill(value=-1,dims=[self.batch_size,self.hist_neighbor_num,self.hist_neighbor_num])


        #mask future position
        seq_mask = tf.range(1,self.max_step+1)
        #input_qa_emb = tf.tile(tf.expand_dims(input_qa_emb,2),[1,1,self.max_step,1])
        similarity_seqs = tf.tile(tf.expand_dims(tf.cast(tf.sequence_mask(seq_mask, self.max_step),
                                    dtype=tf.float32),0),[self.batch_size,1,1])  # [batch_size,ms,ms]
        #mask_seqs = tf.tile(tf.expand_dims(similarity_seqs,-1),[1,1,1,self.embedding_size])
        #input_qa_emb = mask_seqs*input_qa_emb
        q_similarity = q_similarity*similarity_seqs #only history q non zero# [batch_size,ms,ms]

        #setting lower similarity bount
        condition = tf.greater(q_similarity,self.args.att_bound)
        #condition = tf.greater(q_similarity,0.9)
        q_similarity = tf.where(condition,q_similarity,tf.zeros([self.batch_size,self.max_step,self.max_step]))
        q_sim_index = tf.greater(q_similarity,0)#



        self.q_similarity = q_similarity

        temp_hist_index = tf.nn.top_k(q_similarity, self.hist_neighbor_num)[1]# [batch_size,ms,hist_num]
        self.hist_attention_value = tf.nn.top_k(q_similarity, self.hist_neighbor_num)[0]# [batch_size,ms,hist_num]
        #q_similarity[temp_hist_index]>0

        #temp_hist_index = tf.where(self.hist_attention_value>self.args.att_bound,temp_hist_index,-1*tf.ones([self.batch_size,self.max_step,self.hist_neighbor_num],dtype=tf.int32))
        temp_hist_index = tf.where(self.hist_attention_value>0,temp_hist_index,-1*tf.ones([self.batch_size,self.max_step,self.hist_neighbor_num],dtype=tf.int32))





        #temp_hist_index = tf.tile(tf.expand_dims(temp_hist_index[:,:,0],2),[1,1,self.hist_neighbor_num])
        temp_hist_index = tf.reshape(temp_hist_index,[-1,self.max_step*self.hist_neighbor_num])
        #self.temp_hist_index = tf.reshape(temp_hist_index, [-1, self.max_step, self.hist_neighbor_num])

        hist_neighbors_features =tf.reshape(tf.batch_gather(qa_emb, temp_hist_index), [-1, self.max_step, self.hist_neighbor_num, qa_emb.shape[-1]])

        return hist_neighbors_features


    def next_neighbor_sampler(self,aggregate_embedding):

        temp_emb = tf.reshape(aggregate_embedding[1],[-1,self.question_neighbor_num,self.embedding_size])
        temp_emb = tf.transpose(temp_emb, [1, 0, 2])
        temp_emb = tf.transpose(
            tf.gather(temp_emb, tf.random.shuffle(tf.range(tf.shape(temp_emb)[0]))), [1, 0, 2])
        if self.question_neighbor_num>=self.next_neighbor_num:
            next_neighbors_embedding = tf.reshape(temp_emb[:,:self.next_neighbor_num,:],[self.batch_size,self.max_step,self.next_neighbor_num,self.embedding_size])
        else:
            tile_neighbor_embedding = tf.tile(temp_emb,[1, -(-self.next_neighbor_num // tf.shape(temp_emb)[0]), 1])
            next_neighbors_embedding = tf.reshape(tile_neighbor_embedding[:,:self.next_neighbor_num,:],[self.batch_size,self.max_step,self.next_neighbor_num,self.embedding_size])

        return next_neighbors_embedding

    def get_neighbors(self,n_hop, question_index):
        # question_index:[batch_size,seq_len]
        # question_seed = tf.reshape(question_index#[batch_size*seq_len,1]
        seeds = [question_index]  # [[batch_size,seq_len],[batch_size,seq_len,question_neighbor_num],batch_size,seq_len,question_neighbor_num,

        for i in range(n_hop):
            if i % 2 == 0:
                neighbor = tf.reshape(tf.gather(self.question_neighbors, tf.reshape(seeds[i], [-1])),
                                      [-1, self.max_step, self.question_neighbor_num])

            else:
                neighbor = tf.reshape(tf.gather(self.skill_neighbors, tf.reshape(seeds[i], [-1])),
                                      [-1, self.max_step, self.skill_neighbor_num])


            seeds.append(neighbor)  # [batch_size,seq_len,neighbor_num],[batch_size,seq_len,neighbor_num*neighbor_num]


        return seeds

    def aggregate(self, input_neighbors, input_questions_embedding):
        # [[batch_size,seq_len],[batch_size,seq_len,q_neighbor_num],[batch_size,seq_len,q_neighbor_num*s_neighbor_num]]
        sq_neighbor_vectors = []
        for hop_i, neighbors in enumerate(input_neighbors):
            if hop_i % 2 == 0:  # question
                temp_neighbors = tf.reshape(tf.gather(self.feature_embedding, tf.reshape(neighbors, [-1])),
                                            [self.batch_size, self.max_step, -1, self.embedding_size])
                sq_neighbor_vectors.append(temp_neighbors)
            else:  # skill
                temp_neighbors = tf.reshape(tf.gather(self.feature_embedding, tf.reshape(neighbors, [-1])),
                                            [self.batch_size, self.max_step, -1, self.embedding_size])
                sq_neighbor_vectors.append(temp_neighbors)
        aggregators = []
        for i in range(self.n_hop):
            if i == self.n_hop - 1:
                aggregator = self.aggregator_class(self.batch_size, self.max_step, self.embedding_size, act=tf.nn.tanh,
                                                   dropout=self.keep_prob_gnn)
            else:
                aggregator = self.aggregator_class(self.batch_size, self.max_step, self.embedding_size, act=tf.nn.tanh,
                                                   dropout=self.keep_prob_gnn)
            aggregators.append(aggregator)

            # vectors_next_iter = []
            for hop in range(self.n_hop - i):  # aggregate from outside to inside#layer
                if hop % 2 == 0:
                    shape = [self.batch_size, self.max_step, -1, self.question_neighbor_num, self.embedding_size]
                    vector = aggregator(self_vectors=sq_neighbor_vectors[hop],
                                        neighbor_vectors=tf.reshape(sq_neighbor_vectors[hop + 1], shape),
                                        question_embeddings=sq_neighbor_vectors[hop],
                                        )  # [batch_size,seq_len, -1, dim]
                else:
                    shape = [self.batch_size, self.max_step, -1, self.skill_neighbor_num, self.embedding_size]
                    vector = aggregator(self_vectors=sq_neighbor_vectors[hop],
                                        neighbor_vectors=tf.reshape(sq_neighbor_vectors[hop + 1], shape),
                                        question_embeddings=sq_neighbor_vectors[hop],
                                        )  # [batch_size,seq_len, -1, dim]
                # shape = [self.batch_size, self.max_step, -1, self.sample_neighbor_num, self.embedding_size]

                # vectors_next_iter.append(vector)
                sq_neighbor_vectors[hop] = vector
            # sq_neighbor_vectors = vectors_next_iter

        # res = tf.reshape(sq_neighbor_vectors[0], [self.batch_size,self.max_step, self.embedding_size])
        res = sq_neighbor_vectors  # [[batch_size,max_step,-1,embedding_size]...]

        return res, aggregators

    # step on batch
    def train(self, sess, features_answer_index, target_answers, seq_lens, hist_neighbor_index):

        input_feed = {self.features_answer_index: features_answer_index,
                      self.target_answers: target_answers,
                      self.sequence_lens: seq_lens,
                      self.hist_neighbor_index: hist_neighbor_index,
                      self.is_training: True}

        input_feed[self.keep_prob] = self.dropout_keep_probs[0]
        input_feed[self.keep_prob_gnn] = self.dropout_keep_probs[1]
        # input_feed[self.aggregate_keep_prob] = self.dropout_keep_probs[1]

        bin_pred, pred, train_loss, _, aaaa = sess.run(
            [self.binary_pred, self.pred, self.loss, self.train_op, self.flat_target_correctness], input_feed)


        return bin_pred, pred, train_loss

    def evaluate(self, sess, features_answer_index, target_answers, seq_lens, hist_neighbor_index,evaluate_step):

        input_feed = {self.features_answer_index: features_answer_index,
                      self.target_answers: target_answers,
                      self.sequence_lens: seq_lens,
                      self.hist_neighbor_index: hist_neighbor_index,
                      self.is_training: False}

        input_feed[self.keep_prob] = self.dropout_keep_probs[-1]
        input_feed[self.keep_prob_gnn] = self.dropout_keep_probs[-1]

        bin_pred, pred = sess.run([self.binary_pred, self.pred], input_feed)

        return bin_pred, pred


In [ ]:
#data_process.py

import numpy as np
import random
import os
import matplotlib.pyplot as plt


def data_process(args):
    # process data


    train_data_directory = os.path.join(args.data_dir, args.dataset, args.dataset + '_train.csv')
    valid_data_directory = os.path.join(args.data_dir, args.dataset, args.dataset + '_test.csv')
    test_data_directory = os.path.join(args.data_dir, args.dataset, args.dataset + '_test.csv')
    args.train_seqs,train_student_num,train_max_skill_id,train_max_question_id,feature_answer_id = load_data(train_data_directory,args.field_size,args.max_step)
    args.test_seqs,test_student_num,test_max_skill_id,test_max_question_id,_ = load_data(test_data_directory,args.field_size,args.max_step)
    args.valid_seqs,_,_,_,_ = load_data(valid_data_directory,args.field_size,args.max_step)



    print("original test seqs num:%d"%len(args.test_seqs))
    lens = []
    for i in range(len(args.test_seqs)):
        lens.append(len(args.test_seqs[i]))



    lens = []
    for i in range(len(args.test_seqs)):
        lens.append(len(args.test_seqs[i]))





    student_num = train_student_num+test_student_num
    args.skill_num = max(train_max_skill_id,test_max_skill_id)+1
    args.qs_num = max(train_max_question_id,test_max_question_id)+1
    args.question_num = args.qs_num-args.skill_num
    args.feature_answer_size = feature_answer_id+1
    print(args.skill_num)
    print(args.question_num)


    matrix_directory = os.path.join(args.data_dir, args.dataset, args.dataset + '_skill_matrix.txt')
    args.skill_matrix = np.loadtxt(matrix_directory) #multi-skill

    qs_adj_list,interactions = build_adj_list(args.train_seqs,args.test_seqs,args.skill_matrix,args.qs_num)#[[neighbor skill/question] for all qs]]
    args.question_neighbors,args.skill_neighbors = extract_qs_relations(qs_adj_list,args.skill_num,args.qs_num,args.question_neighbor_num,args.skill_neighbor_num)
    # print(args.question_neighbors.shape)#the first s_num rows are 0
    # print(args.skill_neighbors.shape)
    # exit()
    return args

def select_part_seqs(min_len,max_len,seqs):
    temp_seqs = []
    for seq in seqs:
        if len(seq)>=min_len and len(seq)<=max_len:
            temp_seqs.append(seq)

    print("seq num is: %d"%len(temp_seqs))
    return temp_seqs

def build_adj_list(train_seqs,test_seqs,skill_matrix,qs_num):
    #seqs:list-num_students,seq_len,field_size
    #skill_matrix:[num_skill,num_skill]
    #0:skill 1:question
    #question-skill graph
    interactions = 0
    single_skill = []
    adj_list = [[] for _ in range(qs_num)]
    num_skill = skill_matrix.shape[0]

    adj_num = [0 for _ in range(qs_num)]

    for seqs in [train_seqs,test_seqs]:
        for seq in seqs:
            interactions+=len(seq)
            for step in seq:
                adj_list[step[1]] = np.reshape(np.argwhere(skill_matrix[step[0]] == 1),[-1]).tolist()
                adj_num[step[1]] += 1
                for skill_index in np.reshape(np.argwhere(skill_matrix[step[0]] == 1),[-1]).tolist():
                    adj_num[skill_index] += 1
                    if skill_index not in single_skill:
                        single_skill.append(skill_index)
                    if step[1] not in adj_list[skill_index]:
                        adj_list[skill_index].append(step[1])


    # print("average neighbor question num:{}".format(np.sum([len(adj_list[i]) for i in range(num_skill)])/len(single_skill)))
    # print("average neighbor skill num:{}".format(np.sum([len(adj_list[i]) for i in range(num_skill,qs_num)])/(qs_num-num_skill)))


    return adj_list,interactions

def extract_qs_relations(qs_list,s_num,qs_num, q_neighbor_size, s_neighbor_size):
    question_neighbors = np.zeros([qs_num, q_neighbor_size], dtype=np.int32)#the first s_num rows are 0
    skill_neighbors = np.zeros([s_num, s_neighbor_size], dtype=np.int32)
    s_num_dic = {}
    q_num_dic = {}
    for index,neighbors in enumerate(qs_list):
        if index < s_num:  #s
            if len(neighbors) not in q_num_dic:
                q_num_dic[len(neighbors)] = 1
            else:
                q_num_dic[len(neighbors)] +=1
            if len(neighbors) > 0:
                if len(neighbors) >= s_neighbor_size:
                    skill_neighbors[index] = np.random.choice(neighbors, s_neighbor_size, replace=False)
                else:
                    skill_neighbors[index] = np.random.choice(neighbors, s_neighbor_size, replace=True)
        else:#q
            #print(len(neighbors))
            if len(neighbors) not in s_num_dic:
                s_num_dic[len(neighbors)] = 1
            else:
                s_num_dic[len(neighbors)] +=1
            if len(neighbors)>0:
                if len(neighbors) >= q_neighbor_size:
                    question_neighbors[index] = np.random.choice(neighbors, q_neighbor_size,replace=False)
                else:
                    question_neighbors[index] = np.random.choice(neighbors, q_neighbor_size, replace=True)

    #q_num_dic = sorted(q_num_dic.items(), key=lambda d: d[1])
    # print(s_num_dic)
    # print(q_num_dic)
    # exit()
    return question_neighbors,skill_neighbors

def load_data(dataset_path, field_size, max_seq_len):
    seqs = []
    student_id = 0
    max_skill = -1
    max_question = -1
    feature_answer_size = -1
    with open(dataset_path, 'r') as f:
        feature_answer_list = []
        for lineid,line in enumerate(f):
            fields = line.strip().strip(',')
            i = lineid % (field_size+1)
            if i != 0:#i==0 new student==>student seq len
                feature_answer_list.append(list(map(int, fields.split(","))))  #
            if i == 1:
                if max(feature_answer_list[-1]) > max_skill:
                    max_skill = max(feature_answer_list[-1])
            elif i == 2:
                if max(feature_answer_list[-1])>max_question:
                    max_question = max(feature_answer_list[-1])
            elif i == field_size:
                student_id += 1
                if max(feature_answer_list[-1])>feature_answer_size:
                    feature_answer_size = max(feature_answer_list[-1])
                if len(feature_answer_list[0]) > max_seq_len:
                    n_split = len(feature_answer_list[0]) // max_seq_len
                    if len(feature_answer_list[0]) % max_seq_len:
                        n_split += 1
                else:
                    n_split = 1
                for k in range(n_split):
                    # Less than 'seq_len' element remained
                    if k == n_split - 1:
                        end_index = len(feature_answer_list[0])
                    else:
                        end_index = (k + 1) * max_seq_len
                    split_list = []

                    for i in range(len(feature_answer_list)):
                        split_list.append(feature_answer_list[i][k * max_seq_len:end_index])
                        #if i == len(feature_answer_list)-2:#before answer
                            #split_list.append([student_id]*(end_index-k*args.seq_len)) #student id

                    split_list = np.stack(split_list,1).tolist()#[seq_len,field_size]

                    seqs.append(split_list)
                feature_answer_list = []

    return seqs,student_id,max_skill,max_question,feature_answer_size



def pad_sequences(sequences, maxlen=None, dtype='int32', padding='pre', truncating='pre', value=0.):
    lengths = [len(s) for s in sequences]
    nb_samples = len(sequences)
    if maxlen is None:
        maxlen = np.max(lengths)

    # take the sample shape from the first non empty sequence
    # checking for consistency in the main loop below.
    sample_shape = tuple()
   # print(np.shape(sequences))
    for s in sequences:
        if len(s) > 0:
            sample_shape = np.asarray(s).shape[1:]
            break

    x = (np.ones((nb_samples, maxlen) + sample_shape) * value).astype(dtype)

    for idx, s in enumerate(sequences):
        if len(s) == 0:
            continue  # empty list was found
        if truncating == 'pre': # maxlen!=none may need to truncating
            trunc = s[-maxlen:]
        elif truncating == 'post':
            trunc = s[:maxlen+1]
        else:
            raise ValueError('Truncating type "%s" not understood' % truncating)

        # check `trunc` has expected shape
        trunc = np.asarray(trunc, dtype=dtype)
        if trunc.shape[1:] != sample_shape:
            raise ValueError('Shape of sample %s of sequence at position %s is different from expected shape %s' %
                             (trunc.shape[1:], idx, sample_shape))
        if padding == 'post':
            x[idx, :len(trunc)] = trunc
        elif padding == 'pre':
            x[idx, -len(trunc):] = trunc
        else:
            raise ValueError('Padding type "%s" not understood' % padding)
    return x



#select same skill index
def sample_hist_neighbors(seqs_size,max_step,hist_num,skill_index):
    #skill_index:[batch_size,max_step]

    #[batch_size,max_step,M]
    hist_neighbors_index = []

    for i in range(seqs_size):
        seq_hist_index = []
        seq_skill_index = skill_index[i]
        #[max_step,M]
        for j in range(1,max_step):
            same_skill_index = [k for k in range(j) if seq_skill_index[k] == seq_skill_index[j]]

            if hist_num != 0:
                #[0,j] select M
                if len(same_skill_index) >= hist_num:
                    seq_hist_index.append(np.random.choice(same_skill_index,hist_num, replace=False))
                else:
                    if len(same_skill_index)!= 0:
                        seq_hist_index.append(np.random.choice(same_skill_index,hist_num, replace=True))
                    else:
                        seq_hist_index.append(([max_step-1 for _ in range(hist_num)]))
            else:
                seq_hist_index.append([])
        hist_neighbors_index.append(seq_hist_index)
    return hist_neighbors_index


def format_data(seqs, max_step, feature_size, hist_num):

    seqs = seqs
    seq_lens = np.array(list(map(lambda seq: len(seq), seqs)))

    #[batch_size,max_len,feature_size]
    features_answer_index = pad_sequences(seqs,maxlen=max_step, padding='post', value=0)
    target_answers = pad_sequences(np.array([[j[-1] - feature_size for j in i[1:]] for i in seqs]), maxlen=max_step-1, padding='post', value=0)
    skills_index = features_answer_index[:,:,0]
    hist_neighbor_index = sample_hist_neighbors(len(seqs),max_step,hist_num,skills_index)#[batch_size,max_step,M]


    return features_answer_index,target_answers,seq_lens,hist_neighbor_index




class DataGenerator(object):

    def __init__(self, seqs, max_step, batch_size, feature_size, hist_num):#feature_dkt
        np.random.seed(42)
        self.seqs = seqs
        self.max_step = max_step
        self.batch_size = batch_size
        self.batch_i = 0
        self.end = False
        self.feature_size = feature_size
        self.n_batch = int(np.ceil(len(seqs) / batch_size))
        self.hist_num = hist_num




    def next_batch(self):

        batch_seqs = self.seqs[self.batch_i*self.batch_size:(self.batch_i+1)*self.batch_size]
        self.batch_i+=1

        if self.batch_i == self.n_batch:
            self.end = True

        format_data_list = format_data(batch_seqs, self.max_step,self.feature_size,self.hist_num)  # [feature_index,target_answers,sequences_lens,hist_neighbor_index]
        return format_data_list

    def shuffle(self):
        self.pos = 0
        self.end = False
        np.random.shuffle(self.seqs)

    def reset(self):
        self.pos = 0
        self.end = False


# if __name__ == "__main__":
#     seqs = [[[1,0],[2,0],[3,1]],[[2,0],[4,1]]]
#     input_x, target_id, target_correctness, seq_len, max_len = format_data(seqs,1,5)
#     print(input_x)
#     print(target_id)

In [ ]:
#train.py

#from model import GIKT
import tensorflow as tf
import numpy as np
import os
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support, accuracy_score
#from data_process import DataGenerator

def train(args,train_dkt):
    run_config = tf.compat.v1.ConfigProto()
    run_config.gpu_options.allow_growth = True

    with tf.compat.v1.Session(config=run_config) as sess:

        print(args.model)

        model = GIKT(args)

        saver = tf.compat.v1.train.Saver()
        index = 0

        if train_dkt:
            # lr = 0.4
            # lr_decay = 0.92
            sess.run(tf.compat.v1.global_variables_initializer())

            model_dir = save_model_dir(args)

            best_valid_auc = 0

            for epoch in tqdm(range(args.num_epochs)):
                train_generator = DataGenerator(args.train_seqs, args.max_step, batch_size=args.batch_size,
                                                feature_size=args.feature_answer_size - 2,
                                                hist_num=args.hist_neighbor_num)
                valid_generator = DataGenerator(args.valid_seqs, args.max_step, batch_size=args.batch_size,
                                                feature_size=args.feature_answer_size - 2,
                                                hist_num=args.hist_neighbor_num)
                #    assign_lr()
                print("epoch:", epoch)
                # self.assign_lr(self.sess,self.args.lr * self.args.lr_decay ** epoch)
                overall_loss = 0
                train_generator.shuffle()
                preds, binary_preds, targets = list(), list(), list()
                train_step = 0
                while not train_generator.end:
                    train_step += 1

                    [features_answer_index,target_answers,seq_lens,hist_neighbor_index] = train_generator.next_batch()
                    binary_pred, pred, loss = model.train(sess,features_answer_index,target_answers,seq_lens,hist_neighbor_index)

                    overall_loss += loss
                    for seq_idx, seq_len in enumerate(seq_lens):
                        preds.append(pred[seq_idx, 0:seq_len])
                        binary_preds.append(binary_pred[seq_idx, 0:seq_len])
                        targets.append(target_answers[seq_idx, 0:seq_len])
                # print("\r idx:{0}, overall_loss:{1}".format(train_generator.pos, overall_loss)),
                train_loss = overall_loss / train_step
                preds = np.concatenate(preds)
                binary_preds = np.concatenate(binary_preds)
                targets = np.concatenate(targets)
                auc_value = roc_auc_score(targets, preds)
                accuracy = accuracy_score(targets, binary_preds)
                precision, recall, f_score, _ = precision_recall_fscore_support(targets, binary_preds)
                print("\ntrain loss = {0},auc={1}, accuracy={2}".format(train_loss, auc_value, accuracy))
                write_log(args,model_dir,auc_value, accuracy, epoch, name='train_')

                # if epoch == self.args.num_epochs-1:
                #     self.save(epoch)

                # valid
                valid_generator.reset()
                preds, binary_preds, targets = list(), list(), list()
                valid_step = 0
                #overall_loss = 0
                while not valid_generator.end:
                    valid_step += 1
                    [features_answer_index,target_answers,seq_lens,hist_neighbor_index] = valid_generator.next_batch()
                    binary_pred, pred = model.evaluate(sess,features_answer_index,target_answers,seq_lens,hist_neighbor_index,valid_step)
                    #overall_loss += loss
                    for seq_idx, seq_len in enumerate(seq_lens):
                        preds.append(pred[seq_idx, 0:seq_len])
                        binary_preds.append(binary_pred[seq_idx, 0:seq_len])
                        targets.append(target_answers[seq_idx, 0:seq_len])
                # compute metrics
                #valid_loss = overall_loss / valid_step
                preds = np.concatenate(preds)
                binary_preds = np.concatenate(binary_preds)
                targets = np.concatenate(targets)
                auc_value = roc_auc_score(targets, preds)
                accuracy = accuracy_score(targets, binary_preds)
                precision, recall, f_score, _ = precision_recall_fscore_support(targets, binary_preds)
                print("\nvalid auc={0}, accuracy={1}, precision={2}, recall={3}".format(auc_value, accuracy, precision,
                                                                                        recall))


                write_log(args,model_dir,auc_value, accuracy, epoch, name='valid_')

                if auc_value > best_valid_auc:
                    print('%3.4f to %3.4f' % (best_valid_auc, auc_value))
                    best_valid_auc = auc_value
                    best_epoch = epoch
                    #np.save('feature_embedding.npy', feature_embedding)
                    checkpoint_dir = os.path.join(args.checkpoint_dir, model_dir)
                    save(best_epoch,sess,checkpoint_dir,saver)
#                print(model_dir)
                print(model_dir+"\t"+str(best_valid_auc))




        else:
            if self.load():
                print('CKPT loaded')
            else:
                raise Exception('CKPT need')
            test_data_generator = DataGenerator(args.test_seqs, args.max_step, batch_size=args.batch_size,
                                                feature_size=args.feature_answer_size - 2,
                                                hist_num=args.hist_neighbor_num)
            data_generator.reset()

            correct_times = np.zeros(self.num_skills + 1)
            preds, binary_preds, targets = list(), list(), list()
            while not test_data_generator.end:

                [features_answer_index, target_answers, seq_lens, hist_neighbor_index] = valid_generator.next_batch()
                binary_pred, pred = model.evaluate(sess, features_answer_index, target_answers, seq_lens,
                                                   hist_neighbor_index)
                # overall_loss += loss
                for seq_idx, seq_len in enumerate(seq_lens):
                    preds.append(pred[seq_idx, 0:seq_len])
                    binary_preds.append(binary_pred[seq_idx, 0:seq_len])
                    targets.append(target_answers[seq_idx, 0:seq_len])



            preds = np.concatenate(preds)
            binary_preds = np.concatenate(binary_preds)
            targets = np.concatenate(targets)
            auc_value = roc_auc_score(targets, preds)
            accuracy = accuracy_score(targets, binary_preds)
            precision, recall, f_score, _ = precision_recall_fscore_support(targets, binary_preds)
            print("\ntest auc={0}, accuracy={1}, precision={2}, recall={3}".format(auc_value, accuracy, precision,
                                                                                   recall))
            print(model_dir)
            write_log(args, model_dir, auc_value, accuracy, epoch, name='test_')







def save(global_step,sess,checkpoint_dir,saver):
    model_name = 'GIKT'

    if not os.path.exists(checkpoint_dir):
        os.mkdir(checkpoint_dir)
    saver.save(sess, os.path.join(checkpoint_dir, model_name), global_step=global_step)
    print('Save checkpoint at %d' % (global_step))


def save_model_dir(args):
    return '{}_{}_{}lr_{}hop_{}sn_{}qn_{}hn_{}nn_{}_{}bound_{}keep_{}'.format(args.dataset,
                                                args.model,args.lr,args.n_hop,args.skill_neighbor_num,args.question_neighbor_num,args.hist_neighbor_num,\
                                                                     args.next_neighbor_num,args.sim_emb,args.att_bound,args.dropout_keep_probs,args.tag)



def write_log(args,model_dir,auc, accuracy, epoch, name='train_'):
    log_path = os.path.join(args.log_dir, name+model_dir+'.csv')
    if not os.path.exists(log_path):
        log_file = open(log_path, 'w')
        log_file.write('Epoch\tAuc\tAccuracy\n')
    else:
        log_file = open(log_path, 'a')

    log_file.write(str(epoch) + '\t' + str(auc) + '\t' + str(accuracy)  + '\n')
    log_file.flush()


In [ ]:
#main.py

import argparse
import time
import os
import tensorflow as tf
import numpy as np
from numpy.distutils.fcompiler import str2bool
#from data_process import *
#from train import train
import json




def main():
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


    train_dkt = 1
    arg_parser = argparse.ArgumentParser(description="train dkt model")
    arg_parser.add_argument('--data_dir', type=str, default='data')
    arg_parser.add_argument("--log_dir", type=str, default='logs')
    arg_parser.add_argument('--train', type=str2bool, default='t')
    arg_parser.add_argument('--hidden_neurons', type=int, default=[200,100])
    arg_parser.add_argument("--lr", type=float, default=0.001)
    arg_parser.add_argument("--lr_decay", type=float, default=0.92)
    arg_parser.add_argument('--checkpoint_dir', type=str, default='checkpoint')
    arg_parser.add_argument('--dropout_keep_probs', nargs='?', default=[0.6,0.8,1])
    arg_parser.add_argument('--aggregator', type=str, default='sum')
    arg_parser.add_argument('--model', type=str, default='dkt')
    arg_parser.add_argument('--l2_weight', type=float, default=1e-8)
    arg_parser.add_argument('--limit_max_len',type=int,default=200)
    arg_parser.add_argument('--limit_min_len',type=int,default=3)


    arg_parser.add_argument('--dataset', type=str, default='ednet_5000_3')

    arg_parser.add_argument("--field_size", type=int, default=3)
    arg_parser.add_argument("--embedding_size", type=int, default=100)
    arg_parser.add_argument("--max_step", type=int, default=200)
    arg_parser.add_argument("--input_trans_size", type=int, default=100)
    arg_parser.add_argument("--batch_size", type=int, default=32)
    arg_parser.add_argument("--select_index", type=int, default=[0,1, 2])
    arg_parser.add_argument('--num_epochs', type=int, default=150)
    arg_parser.add_argument('--n_hop', type=int, default=3)
    arg_parser.add_argument('--skill_neighbor_num', type=int, default=10)
    arg_parser.add_argument('--question_neighbor_num', type=int, default=4)
    arg_parser.add_argument('--hist_neighbor_num', type=int, default=0)  # history neighbor num
    arg_parser.add_argument('--next_neighbor_num', type=int, default=4)  # next neighbor num

    arg_parser.add_argument('--att_bound', type=float, default=0.5)#filtring irralate emb in topk selection
    arg_parser.add_argument('--sim_emb', type=str, default='skill_emb')#filtring irralate emb in topk selection



    args = arg_parser.parse_args()
    #args.dataset = dataset
    print(args.model)
    tag_path = os.path.join("%s_tag.txt"%args.dataset)
    tag = time.time()
    args.tag = tag

    config_name = 'logs/%f_config.json' % tag
    config = {}
    for k,v in vars(args).items():
        config[k] = vars(args)[k]

    jsObj = json.dumps(config)

    fileObject = open(config_name, 'w')
    fileObject.write(jsObj)
    fileObject.close()
    print(config)
    args = data_process(args)

    train(args,train_dkt)

    log_file = open(tag_path, 'w')
    log_file.write(str(tag))

if __name__ == "__main__":
    main()


usage: ipykernel_launcher.py [-h] [--data_dir DATA_DIR] [--log_dir LOG_DIR]
                             [--train TRAIN] [--hidden_neurons HIDDEN_NEURONS]
                             [--lr LR] [--lr_decay LR_DECAY]
                             [--checkpoint_dir CHECKPOINT_DIR]
                             [--dropout_keep_probs [DROPOUT_KEEP_PROBS]]
                             [--aggregator AGGREGATOR] [--model MODEL]
                             [--l2_weight L2_WEIGHT]
                             [--limit_max_len LIMIT_MAX_LEN]
                             [--limit_min_len LIMIT_MIN_LEN]
                             [--dataset DATASET] [--field_size FIELD_SIZE]
                             [--embedding_size EMBEDDING_SIZE]
                             [--max_step MAX_STEP]
                             [--input_trans_size INPUT_TRANS_SIZE]
                             [--batch_size BATCH_SIZE]
                             [--select_index SELECT_INDEX]
                             [--

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
